In [1]:
import wave
import struct
import time
import numpy as np
from scipy.fft import fft
import matplotlib.pyplot as plt
import serial
import threading
import queue
import pyaudio
import matplotlib.animation as animation
%matplotlib tk
CHUNK = 1024*4 # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

# Set up serial communication with Arduino
arduino = serial.Serial('COM4', 9600)
time.sleep(2)  # Wait for Arduino to initialize

# Create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

wav_file = wave.open(r'C:\Users\rajka\Downloads\My Song 3 1.wav', 'rb')
print('Playback started')
# Stream object to get data from microphone
stream = p.open(
    format=p.get_format_from_width(wav_file.getsampwidth()),
    channels=wav_file.getnchannels(),
    rate=wav_file.getframerate(),
    input=False,
    output=True,
    frames_per_buffer=CHUNK
)

# Variables for plotting
x = np.arange(0, 2 * CHUNK, 2)
xf = np.linspace(0, RATE, CHUNK)

# Create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# Create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# Format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# Format spectrum axes
ax2.set_xlim(20, RATE / 2)
chosen_frequencies = [100, 250, 500, 750, 10000, 2500]
chosen_indices = [np.abs(xf - freq).argmin() for freq in chosen_frequencies]



# Read the audio data from the .wav file
data = wav_file.readframes(CHUNK)

# Create a queue to exchange data between threads
data_queue = queue.Queue()

# Variable to control playback
is_playing = True


# Function to process audio data
def process_audio():
    while is_playing:
        data_int = np.array(struct.unpack(f"{2 * CHUNK}h", data), dtype=np.int16) + 128
        data_np = np.array(data_int, dtype='b')[::2] + 128
        yf = fft(data_int)

        data_queue.put((data_np, yf))


# Function to update the plot
def update_plot():
    while is_playing:
        try:
            data_np, yf = data_queue.get(timeout=1)
        except queue.Empty:
            continue

        line.set_ydata(data_np)

        for freq, index in zip(chosen_frequencies, chosen_indices):
            value = np.abs(yf[index]) / (128 * CHUNK)
            print(f"Frequency: {freq} Hz, Value: {value}")
            message = f"{freq}:{value}\n".encode()
            arduino.write(message)

        line_fft.set_ydata(np.abs(yf[0:CHUNK]) / (128 * CHUNK))
        return line, line_fft

        # fig.canvas.draw()
        # fig.canvas.flush_events()

ani = animation.FuncAnimation(fig, update_plot, frames=None, interval=0, cache_frame_data=False)

# Start audio processing thread
audio_thread = threading.Thread(target=process_audio)
audio_thread.start()

# Start plot updating thread
plot_thread = threading.Thread(target=update_plot)
plot_thread.start()

while len(data) > 0:
    stream.write(data)

    # Read the next chunk of audio data from the .wav file
    data = wav_file.readframes(CHUNK)

plt.show(block=True)

# Wait for audio processing to finish
audio_thread.join()

# Wait for plot updating to finish
plot_thread.join()

# Close the .wav file and serial connection
wav_file.close()
arduino.close()

print('Playback finished')


Playback started
Frequency: 100 Hz, Value: 26.469568025748806
Frequency: 250 Hz, Value: 2.2270414294132745
Frequency: 500 Hz, Value: 2.821384206021317
Frequency: 750 Hz, Value: 1.5857675910438704
Frequency: 10000 Hz, Value: 0.052736075663198344
Frequency: 2500 Hz, Value: 0.3752636504998959


KeyboardInterrupt: 

Traceback (most recent call last):
  File "c:\Users\rajka\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\cbook\__init__.py", line 304, in process
    func(*args, **kwargs)
  File "c:\Users\rajka\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\animation.py", line 904, in _start
    self._init_draw()
  File "c:\Users\rajka\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\animation.py", line 1748, in _init_draw
    self._draw_frame(frame_data)
  File "c:\Users\rajka\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\animation.py", line 1767, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: update_plot() takes 0 positional arguments but 1 was given


In [2]:
import serial

# Open the serial port
port = serial.Serial('COM4', 9600)  # Replace with the appropriate port and baud rate

while True:
    # Read input from the keyboard
    data = input("Enter a value: ")

    # Send the data to the serial port
    port.write(data.encode())

    # Wait for a response from the serial monitor
    response = port.readline().decode().strip()

    # Print the response
    print("Received:", response)

# Close the serial port
port.close()

KeyboardInterrupt: 